In [1]:
import requests
# pretty print is used to print the output in the console in an easy to read format
from pprint import pprint
from datetime import datetime
from web3 import Web3
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import numpy as np

In [2]:
# transaction ids has to be checked in etherscan
node_url = "https://alien-sparkling-gadget.discover.quiknode.pro/ea1f5f2f5471b3a0525fa85dbfcfa27a5f90e25d/"
web3 = Web3(Web3.HTTPProvider(node_url))
tx = web3.eth.get_transaction('0xffe16a6ba847b9d189b6bb28686d1418bc1f4ef150800e4c48b5c02fe3555797')
block = tx.blockNumber
ts = web3.eth.get_block(block).timestamp
datetime.utcfromtimestamp(ts)

datetime.datetime(2023, 5, 21, 14, 41, 23)

In [3]:
from web3 import Web3
from web3 import AsyncHTTPProvider
from web3.eth import AsyncEth
import asyncio
import json
import math
import nest_asyncio
nest_asyncio.apply()


In [4]:
nodes = ['https://mainnet.infura.io/v3/174e045511c742af9cbe23f6bd053402',
         'https://mainnet.infura.io/v3/0731bcfffdb844daa0972c90e98da4e6',
         'https://mainnet.infura.io/v3/6d0c853857a249878418776d98bf732a']

In [5]:
# Define providers
w3 = Web3(Web3.HTTPProvider(nodes[0]))
providers = []
providersAsync = []
for node in nodes:
    providers.append(Web3.HTTPProvider(node))
    providersAsync.append(Web3(AsyncHTTPProvider(node), modules={"eth": (AsyncEth)}))

In [6]:
# Read factory contract addresses
# Uniswap V2 factory contract address
# contract_address = '0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f'
# SushiSwap factory contract address
# contract_address = '0xC0AEe478e3658e2610c5F7A4A2E1777cE9e4f2Ac'
with open("/home/ildefons/memeprj/Articles-mev-bot-series/Part 3/FactoriesV2.json", "r") as f:
    factories = json.load(f)

In [7]:
# Define the contract ABI
factory_abi = [
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": True,
                "internalType": "address",
                "name": "token0",
                "type": "address",
            },
            {
                "indexed": True,
                "internalType": "address",
                "name": "token1",
                "type": "address",
            },
            {
                "indexed": False,
                "internalType": "address",
                "name": "pair",
                "type": "address",
            },
            {
                "indexed": False,
                "internalType": "uint256",
                "name": "",
                "type": "uint256",
            },
        ],
        "name": "PairCreated",
        "type": "event",
    }
]

In [8]:
def getPairEvents(contract, fromBlock, toBlock):
    toBlockPrime = toBlock
    fetchCount = 0

    # Then, recursively fetch events in smaller time intervals
    def getEventsRecursive(contract, _from, _to):
        try:
            events = (
                contract.events.PairCreated()
                .create_filter(fromBlock=_from, toBlock=_to)
                .get_all_entries()
            )
            print("Found ", len(events), " events between blocks ", _from, " and ", _to)
            nonlocal fetchCount
            fetchCount += len(events)
            return events
        except ValueError:
            print("Too many events found between blocks ", _from, " and ", _to)
            midBlock = (_from + _to) // 2
            return getEventsRecursive(contract, _from, midBlock) + getEventsRecursive(
                contract, midBlock + 1, _to
            )

    return getEventsRecursive(contract, fromBlock, toBlockPrime)

In [9]:
# Fetch list of pools for each factory contract
pairDataList = []
for factoryName, factoryData in factories.items():
    events = getPairEvents(
        w3.eth.contract(address=factoryData["factory"], abi=factory_abi),
        0,
        w3.eth.block_number,
    )
    print(f"Found {len(events)} pools for {factoryName}")
    for e in events:
        pairDataList.append(
            {
                "token0": e["args"]["token0"],
                "token1": e["args"]["token1"],
                "pair": e["args"]["pair"],
                "factory": factoryName,
            }
        )

Too many events found between blocks  0  and  17756543
Found  0  events between blocks  0  and  8878271
Too many events found between blocks  8878272  and  17756543
Too many events found between blocks  8878272  and  13317407
Too many events found between blocks  8878272  and  11097839
Found  0  events between blocks  8878272  and  9988055
Too many events found between blocks  9988056  and  11097839
Found  2965  events between blocks  9988056  and  10542947
Too many events found between blocks  10542948  and  11097839
Found  5117  events between blocks  10542948  and  10820393
Found  8708  events between blocks  10820394  and  11097839
Too many events found between blocks  11097840  and  13317407
Too many events found between blocks  11097840  and  12207623
Too many events found between blocks  11097840  and  11652731
Found  7971  events between blocks  11097840  and  11375285
Found  3611  events between blocks  11375286  and  11652731
Found  4101  events between blocks  11652732  and 

In [11]:
# %%
WETH = "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2"
pair_pool_dict = {}
for pair_object in pairDataList:
    # Check for ETH (WETH) in the pair.
    pair = (pair_object['token0'], pair_object['token1'])
    if WETH not in pair:
        continue

    # Make sure the pair is referenced in the dictionary.
    if pair not in pair_pool_dict:
        pair_pool_dict[pair] = []

    # Add the pool to the list of pools that trade this pair.
    pair_pool_dict[pair].append(pair_object)

# Create the final dictionnary of pools that will be traded on.
pool_dict = {}
for pair, pool_list in pair_pool_dict.items():
    if len(pool_list) >= 2:
        pool_dict[pair] = pool_list

In [12]:
# %%
# Helper functions for calculating the optimal trade size
# Output of a single swap
def swap_output(x, a, b, fee=0.003):
    return b * (1 - a/(a + x*(1-fee)))

# Gross profit of two successive swaps
def trade_profit(x, reserves1, reserves2, fee=0.003):
    a1, b1 = reserves1
    a2, b2 = reserves2
    return swap_output(swap_output(x, a1, b1, fee), b2, a2, fee) - x

# Optimal input amount
def optimal_trade_size(reserves1, reserves2, fee=0.003):
    a1, b1 = reserves1
    a2, b2 = reserves2
    return (math.sqrt(a1*b1*a2*b2*(1-fee)**4 * (b1*(1-fee)+b2)**2) - a1*b2*(1-fee)*(b1*(1-fee)+b2)) / ((1-fee) * (b1*(1-fee) + b2))**2

In [43]:
# by the time being I can skip the solidity that reads reserves from pairs and use an existing contract
queryContractAddress = "0x6c618c74235c70DF9F6AD47c6b5E9c8D3876432B"
# The following parts compile and upload the contract code to a local node to experiment without paying gas fees. Later however, we will use the contract already deployed on the mainnet.
# Be careful as the code will use the abi of the compiled contract, which will be different from the abi of the deployed contract if you make changes to it.
contractContent = """
//SPDX-License-Identifier: UNLICENSED
pragma solidity 0.8;

pragma experimental ABIEncoderV2;

interface IUniswapV2Pair {
    function token0() external view returns (address);
    function token1() external view returns (address);
    function getReserves() external view returns (uint112 reserve0, uint112 reserve1, uint32 blockTimestampLast);
}

// Batch query contract
contract UniswapFlashQuery {
    function getReservesByPairs(IUniswapV2Pair[] calldata _pairs) external view returns (uint256[3][] memory) {
        uint256[3][] memory result = new uint256[3][](_pairs.length);
        for (uint i = 0; i < _pairs.length; i++) {
            (result[i][0], result[i][1], result[i][2]) = _pairs[i].getReserves();
        }
        return result;
    }
    function getReservesByPairsYul(IUniswapV2Pair[] calldata _pairs) external view returns (bytes32[] memory) {
        bytes32[] memory result = new bytes32[](_pairs.length * 3);

        assembly {
            let size := 0x60 // Size of the return data (reserve0, reserve1, blockTimestampLast)
            let callData := mload(0x40) // Allocate memory for the function selector
            mstore(callData, 0x0902f1ac00000000000000000000000000000000000000000000000000000000) // 4-byte function selector of the getReserves() function
            
            // Update the free memory pointer
            mstore(0x40, add(callData, 0x04))
            
            // let pairsCount := shr(0xe0, calldataload(sub(_pairs.offset, 0x20))) // Get the length of the _pairs array
            let pairsCount := _pairs.length

            for { let i := 0 } lt(i, pairsCount) { i := add(i, 1) } {
                // Load the pair address from the calldata
                let pair := calldataload(add(_pairs.offset, mul(i, 0x20)))
                    
                // Call the getReserves() function with preallocated memory for function selector
                let success := staticcall(gas(), pair, callData, 0x04, add(add(result, 0x20),mul(i, size)), size)
                if iszero(success) {
                    revert(0x00, 0x00)
                }
            }

            // Update the free memory pointer
            mstore(0x40, add(mload(0x40), mul(pairsCount, size)))
        }

        return result;
    }
}
"""
import solcx
version = "0.8.0"
filename = "UniswapFlashQuery.sol"
Output = "ir"
solcx.install_solc('0.8.0')   #ILDE

compiled_sol = solcx.compile_standard(
    {
        "language": "Solidity",
        "sources": {filename: {"content": contractContent}},
        "settings": {
            "outputSelection": {
                "*": items(){"*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap",Output]}
            }
        },
    }#,
    #solc_version=version,
)

name = filename.split('.')[0]
res_bytecode = compiled_sol["contracts"][filename][name]["evm"]["bytecode"]["object"]
queryAbi = compiled_sol["contracts"][filename][name]["abi"]
# print(compiled_sol["contracts"][filename][name][Output])

# Export the bytecode
print(f"Bytecode: {res_bytecode[:100]}...")

# This code splits the request into chunks that are sent concurrently to the node.
import asyncio
from web3 import AsyncHTTPProvider
from web3.eth import AsyncEth
import time
# If you test this code in a Jupyter notebook, sligh modifications are needed like nest_asyncio.apply() (google for more info)
# [...]

# Create a function that takes a list of pair data, and returns a list of reserves for each pair
async def getReservesAsync(pairs, chunkSize=1000):
    # Create an async web3 provider instance
    w3Async = Web3(AsyncHTTPProvider(node), modules={"eth": (AsyncEth)})

    # Create contract object
    queryContract = w3Async.eth.contract(address=queryContractAddress, abi=queryAbi)

    # Create a list of chunks of pair addresses
    chunks = [[pair for pair in pairs[i:i + chunkSize]] for i in range(0, len(pairs), chunkSize)]

    # Gather all the async tasks
    tasks = [queryContract.functions.getReservesByPairs(pairs).call() for pairs in chunks]

    print(len(tasks))
    # Run the tasks in parallel
    results = await asyncio.gather(*tasks)

    return results

providerList = [Web3(AsyncHTTPProvider(uri), modules={'eth': (AsyncEth)}) for uri in nodes]

async def getReservesParallel(pairs, providers, chunkSize=1000):
    # Create the contract objects
    contracts = [provider.eth.contract(address=queryContractAddress, abi=queryAbi) for provider in providers]

    # Create a list of chunks of pair addresses
    chunks = [[pair for pair in pairs[i:i + chunkSize]] for i in range(0, len(pairs), chunkSize)]

    # Assign each chunk to a provider in a round-robin fashion
    tasks = [contracts[i % len(contracts)].functions.getReservesByPairs(pairs).call() for i, pairs in enumerate(chunks)]
    print(len(tasks))

    # Run the tasks in parallel
    results = await asyncio.gather(*tasks)
    
    # Flatten the results
    results = [item for sublist in results for item in sublist]

    return results

solc, the solidity compiler commandline interface
Version: 0.8.0+commit.c7dfd78e.Linux.g++
Bytecode: 608060405234801561001057600080fd5b50610a64806100206000396000f3fe608060405234801561001057600080fd5b50...


In [62]:
# Fetch the reserves of each pool in pool_dict
to_fetch = [] # List of pool addresses for which reserves need to be fetched.
for pair, pool_list in pool_dict.items():
    for pair_object in pool_list:
        to_fetch.append(pair_object["pair"]) # Add the address of the pool
print(f"Fetching reserves of {len(to_fetch)} pools...")
# getReservesParallel() is from article 2 in the MEV bot series
reserveList = asyncio.get_event_loop().run_until_complete(getReservesParallel(to_fetch, providersAsync))
#reserveList = asyncio.get_event_loop().run_until_complete(getReservesAsync(to_fetch))

Fetching reserves of 3151 pools...
4


In [63]:
len(reserveList)

3151

In [64]:
#---->IMHERE Build list of trading opportunities
index = 0
opps = []
for pair, pool_list in pool_dict.items():
    # Store the reserves in the pool objects for later use
    for pair_object in pool_list:
        pair_object["reserves"] = reserveList[index]
        index += 1

    # Iterate over all the pools of the pair
    for poolA in pool_list:
        for poolB in pool_list:
            # Skip if it's the same pool
            if poolA["pair"] == poolB["pair"]:
                continue

            # Skip if one of the reserves is 0 (division by 0)
            if 0 in poolA["reserves"] or 0 in poolB["reserves"]:
                continue

            # Re-order the reserves so that WETH is always the first token
            if poolA["token0"] == WETH:
                res_A = (poolA["reserves"][0], poolA["reserves"][1])
                res_B = (poolB["reserves"][0], poolB["reserves"][1])
            else:
                res_A = (poolA["reserves"][1], poolA["reserves"][0])
                res_B = (poolB["reserves"][1], poolB["reserves"][0])
            
            # Compute value of optimal input through the formula
            x = optimal_trade_size(res_A, res_B)

            # Skip if optimal input is negative (the order of the pools is reversed)
            if x < 0:
                continue

            # Compute gross profit in Wei (before gas cost)
            profit = trade_profit(x, res_A, res_B)


            # Store details of the opportunity. Values are in ETH. (1e18 Wei = 1 ETH)
            opps.append({
                "profit": profit / 1e18,
                "input": x / 1e18,
                "pair": pair,
                "poolA": poolA,
                "poolB": poolB,
            })

print(f"Found {len(opps)} opportunities.")

Found 1834 opportunities.


In [49]:
index


4